In [61]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import datetime
from vars import *

In [62]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator


dataset_path = dataset_path

log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(200, 200),
    batch_size=64)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(200, 200),
    batch_size=64)

train_dataset = train_dataset.map(lambda x, y: (x/255, y))
validation_dataset = validation_dataset.map(lambda x, y: (x/255, y))


AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)


# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2) 


# train_generator = train_datagen.flow_from_directory(
#     dataset_path,
#     target_size=(150, 150),
#     batch_size=64,
#     class_mode='categorical', 
#     subset='training') 


# validation_generator = train_datagen.flow_from_directory(
#     dataset_path,
#     target_size=(150, 150),
#     batch_size=32,
#     class_mode='categorical',
#     subset='validation')

Found 9152 files belonging to 100 classes.
Using 7322 files for training.
Found 9152 files belonging to 100 classes.
Using 1830 files for validation.


In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom, RandomCrop

model = Sequential([
    RandomFlip("horizontal", input_shape=(200, 200, 3)),
    RandomRotation(0.1),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dense(100, activation='softmax')
])

model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_11 (RandomFlip)  (None, 200, 200, 3)      0         
                                                                 
 random_rotation_11 (RandomR  (None, 200, 200, 3)      0         
 otation)                                                        
                                                                 
 conv2d_35 (Conv2D)          (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d_35 (MaxPoolin  (None, 99, 99, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 48, 48, 64)     

In [64]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [65]:
history = model.fit(
    train_dataset,
    epochs=12,
    validation_data=validation_dataset)

Epoch 1/6


115/115 [==============================] - 114s 532ms/step - loss: 4.6093 - accuracy: 0.0100 - val_loss: 4.6273 - val_accuracy: 0.0126
Epoch 2/6
115/115 [==============================] - 47s 409ms/step - loss: 4.5478 - accuracy: 0.0201 - val_loss: 4.5019 - val_accuracy: 0.0311
Epoch 3/6
115/115 [==============================] - 43s 374ms/step - loss: 4.4032 - accuracy: 0.0388 - val_loss: 4.4205 - val_accuracy: 0.0432
Epoch 4/6
115/115 [==============================] - 44s 387ms/step - loss: 4.2164 - accuracy: 0.0642 - val_loss: 4.4053 - val_accuracy: 0.0464
Epoch 5/6
115/115 [==============================] - 45s 388ms/step - loss: 4.0100 - accuracy: 0.0982 - val_loss: 4.2836 - val_accuracy: 0.0607
Epoch 6/6
115/115 [==============================] - 43s 370ms/step - loss: 3.7979 - accuracy: 0.1310 - val_loss: 4.2473 - val_accuracy: 0.0727
